In [ ]:
import os
import urllib

In [ ]:
import folium
import gmsh
import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
import pyproj

In [ ]:
import femlium

Auxiliary function to get a `folium` `Map` close to Lake Garda.

In [ ]:
def get_garda_geo_map(boundary_icons: bool = False) -> folium.Map:
    """Get a map close to Lake Garda, and possibly add some boundary markers."""
    # Add map close to Lake Garda
    geo_map = folium.Map(location=[45.6389113, 10.7521368], zoom_start=10.3)

    # Add markers
    if boundary_icons:
        location_markers = {
            "Sarca": [45.87395405, 10.87087005],
            "Mincio": [45.43259035, 10.7007715]
        }
        location_colors = {
            "Sarca": "red",
            "Mincio": "green"
        }

        for key in location_markers.keys():
            folium.Marker(
                location=location_markers[key],
                tooltip=key,
                icon=folium.Icon(color=location_colors[key])
            ).add_to(geo_map)

    # Return folium map
    return geo_map

In [ ]:
get_garda_geo_map()

Read the file containing an approximation of the boundary of the lake from csv.

In [ ]:
boundary_csv_filename = "garda.csv"
if not os.path.isfile(boundary_csv_filename):
    os.makedirs("data", exist_ok=True)
    boundary_csv_url = (
        "https://raw.githubusercontent.com/FEMlium/FEMlium/main/"
        "tutorials/01_introduction/data/garda.csv")
    with urllib.request.urlopen(boundary_csv_url) as response, \
            open(boundary_csv_filename, "wb") as boundary_csv_file:
        boundary_csv_file.write(response.read())

In [ ]:
points_and_markers = np.loadtxt("garda.csv", delimiter=",", skiprows=1)
points = points_and_markers[:, 0:2]
vertex_markers = points_and_markers[:, 2].astype(np.int64)
segment_markers = np.array(
    [min(vertex_markers[v], vertex_markers[v + 1]) for v in range(points.shape[0] - 1)], dtype=np.int64)

Plot the domain boundary using `matplotlib`.

In [ ]:
fig = plt.figure(figsize=(12, 12))
fig.gca().plot(points[:, 0], points[:, 1])
fig.gca().axis("equal")

Read a further file containing a curve that corresponds to approximately 1000 m from the shoreline.

In [ ]:
interface_csv_filename = "garda_interface.csv"
if not os.path.isfile(interface_csv_filename):
    os.makedirs("data", exist_ok=True)
    interface_csv_url = (
        "https://raw.githubusercontent.com/FEMlium/FEMlium/main/"
        "tutorials/01_introduction/data/garda_interface.csv")
    with urllib.request.urlopen(interface_csv_url) as response, \
            open(interface_csv_filename, "wb") as interface_csv_file:
        interface_csv_file.write(response.read())

In [ ]:
points_inner = np.loadtxt("garda_interface.csv", delimiter=",", skiprows=1)

Plot the two curves with `matplotlib`.

In [ ]:
fig = plt.figure(figsize=(12, 12))
fig.gca().plot(points[:, 0], points[:, 1], "blue")
fig.gca().plot(points_inner[:, 0], points_inner[:, 1], "orange")
fig.gca().axis("equal")

Define a `pyproj` `Transformer` to map between different reference systems, because the points read from file are stored a $(x, y)$ pairs in the EPSG32632 reference system, while the map produced by `folium` is based on (latitude, longitude) pairs in the EPSG4326 reference system.

In [ ]:
transformer = pyproj.Transformer.from_crs("epsg:32632", "epsg:4326", always_xy=True)

We define a domain plotter `femlium.DomainPlotter`.

In [ ]:
domain_plotter = femlium.DomainPlotter(transformer)

We use the `domain_plotter` to draw the boundary of what will be the computational domain on top of the geographic map.

In [ ]:
geo_map = get_garda_geo_map()
domain_plotter.add_domain_to(geo_map, points)
geo_map

We may change the color and the weight of the line.

In [ ]:
geo_map = get_garda_geo_map()
domain_plotter.add_domain_to(geo_map, points, colors="blue", weights=3)
geo_map

Furthermore, we may set the colors and the weights of the boundary representation to depend on the markers associated to each segment. Such markers are typically used to set boundary labels for boundary conditions.

In [ ]:
geo_map = get_garda_geo_map(boundary_icons=True)
colors = {
    0: "blue",
    1: "red",
    2: "green"
}
weights = {
    0: 2,
    1: 5,
    2: 5
}
domain_plotter.add_domain_to(geo_map, points, segment_markers, colors=colors, weights=weights)
geo_map

We may also plot two curves on the same map.

In [ ]:
geo_map = get_garda_geo_map()
domain_plotter.add_domain_to(geo_map, points, colors="blue", weights=3)
domain_plotter.add_domain_to(geo_map, points_inner, colors="orange", weights=2)
geo_map

We then use `gmsh` to generate a triangular mesh, and save it to file.

In [ ]:
def generate_garda_mesh(
    points: npt.NDArray[np.float64], segment_markers: npt.NDArray[np.int64],
    points_inner: npt.NDArray[np.float64], h: float
) -> None:
    """Generate an unstructured mesh of the Lake."""
    # Initialize gmsh
    gmsh.initialize()
    # Add model
    gmsh.model.add("garda")
    # Add points associated to outer boundary
    num_points = points.shape[0] - 1  # exclude the last point, because it is a copy of the first one
    for p in range(num_points):
        gmsh.model.geo.addPoint(points[p, 0], points[p, 1], 0.0)
    # Add points associted to inner interface
    num_points_inner = points_inner.shape[0] - 1  # exclude the last point, because it is a copy of the first one
    for p in range(num_points_inner):
        gmsh.model.geo.addPoint(points_inner[p, 0], points_inner[p, 1], 0.0)
    # Add segments associated to outer boundary
    lines = list()
    boundary_labels = {
        1: list(),  # shore
        2: list(),  # inflow
        3: list()   # outflow
    }
    for p in range(num_points):
        point_p = p + 1
        if point_p < num_points:
            point_p_next = point_p + 1
        else:
            point_p_next = 1
        segment = gmsh.model.geo.addLine(point_p, point_p_next)
        lines.append(segment)
        label = 1 + segment_markers[p]
        label = round(label)
        boundary_labels[label].append(segment)
    # Add segments associated to inner interface
    lines_inner = list()
    for p in range(num_points_inner):
        point_p = num_points + p + 1
        if point_p < num_points + num_points_inner:
            point_p_next = point_p + 1
        else:
            point_p_next = num_points + 1
        segment = gmsh.model.geo.addLine(point_p, point_p_next)
        lines_inner.append(segment)
    # Add curve loops
    boundary = gmsh.model.geo.addCurveLoop(lines)
    interface = gmsh.model.geo.addCurveLoop(lines_inner)
    # Add surfaces
    near_shore = gmsh.model.geo.addPlaneSurface([boundary, interface])
    far_from_shore = gmsh.model.geo.addPlaneSurface([interface])
    # Synchronize
    gmsh.model.geo.synchronize()
    # Assign boundary labels
    for (label, lines) in boundary_labels.items():
        gmsh.model.addPhysicalGroup(1, lines, label)
    # Assign subdomain labels
    gmsh.model.addPhysicalGroup(2, [near_shore], 1)
    gmsh.model.addPhysicalGroup(2, [far_from_shore], 2)
    # We now define a Distance field, to measure the distance from the boundary
    gmsh.model.mesh.field.add("Distance", 1)
    gmsh.model.mesh.field.setNumbers(1, "PointsList", np.arange(1, num_points + 1))
    gmsh.model.mesh.field.setNumbers(1, "CurvesList", lines)
    gmsh.model.mesh.field.setNumber(1, "NumPointsPerCurve", 1)
    # We then define a Threshold field, which uses the return value of the
    # Distance field 1 in order to define a simple change in element size
    # depending on the computed distances
    #
    # SizeMax -                     /------------------
    #                              /
    #                             /
    #                            /
    # SizeMin -o----------------/
    #          |                |    |
    #        Point         DistMin  DistMax
    gmsh.model.mesh.field.add("Threshold", 2)
    gmsh.model.mesh.field.setNumber(2, "InField", 1)
    gmsh.model.mesh.field.setNumber(2, "SizeMin", h)
    gmsh.model.mesh.field.setNumber(2, "SizeMax", 10 * h)
    gmsh.model.mesh.field.setNumber(2, "DistMin", 600)
    gmsh.model.mesh.field.setNumber(2, "DistMax", 6000)
    # The mesh size should be derived from the second field, i.e. the Distance field
    gmsh.model.mesh.field.setAsBackgroundMesh(2)
    gmsh.option.setNumber("Mesh.MeshSizeExtendFromBoundary", 0)
    gmsh.option.setNumber("Mesh.MeshSizeFromPoints", 0)
    gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 0)
    # Generate mesh
    gmsh.model.mesh.generate(2)
    # Write out the mesh in gmsh format
    msh_path = "garda.msh"
    if os.path.exists(msh_path):
        os.remove(msh_path)
    gmsh.write(msh_path)
    # Finalize
    gmsh.finalize()

In [ ]:
generate_garda_mesh(points, segment_markers, points_inner, 300)

The mesh will be read in and displayed in one of the FEM backends available in `femlium`. See the notebooks in the parent folder.